In [1]:
import tensorflow as tf
import numpy as np


In [2]:
sess = tf.Session()

In [3]:
# Builds The Network
def buildNetwork(inputs, batch_size, NUM_CLASSES=2,keep_prob=1.0):
    def conv_layer(x, num_channels_out, spatial_stride=1,filterSize=3,name=None):
        """ Layer for 3x3 convolutions.

        Args:
          x: A 4-D float32 Tensor with shape [num_images, height, width, num_channels].
          num_channels_out: An integer. The number of output channels we'll compute
            (with one convolutional filter per output channel).
          spatial_stride: A positive integer. If this is 1, we obtain a typical
            convolution; if 2, we'll have one output pixel for every *two* input
            pixels; and so on.

        Returns:
          A 4-D float32 Tensor with shape [num_images, new_height, new_width, num_channels_out].
        """
        num_channels_in = x.get_shape().as_list()[-1]
        conv_strides = [1, spatial_stride, spatial_stride, 1]
        W_shape = [filterSize, filterSize, num_channels_in, num_channels_out]
        W = tf.Variable(tf.truncated_normal(
            W_shape,
            mean=0.0,
            stddev=5e-2,
            dtype=tf.float32,
            seed=None,
            name=None
        ))
        b = tf.Variable(tf.zeros([num_channels_out]))
        conv = tf.nn.conv2d(x, W, conv_strides, 'SAME',name=name)
        conv_with_bias = conv + b
        return conv_with_bias

    def linear_layer(x, num_outputs,name=None):
        """ A simple linear layer.

        Args:
          x: A 2-D float32 Tensor with shape [num_images, num_inputs]. (Each
            image is represented by a vector with dimensionality num_inputs.)
          num_outputs: An integer.

        Returns:
          A 2-D float32 Tensor with shape [num_images, num_outputs].
        """
        num_inputs = x.get_shape().as_list()[-1]
        W_shape = [num_inputs, num_outputs]
        W = tf.Variable(tf.truncated_normal(
            W_shape,
            mean=0.0,
            stddev=5e-2,
            dtype=tf.float32,
            seed=None,
            name=None
        ))
        b = tf.Variable(tf.zeros([num_outputs]))
        ret = tf.nn.xw_plus_b(x, W, b,name=name)
        return ret
        
    x = conv_layer(inputs, num_channels_out=32,filterSize=5,name="Conv1_5x5")
    x = tf.nn.relu(x,name="relu1")
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                     padding='SAME', name='pool1')
  #  x = tf.nn.lrn(x, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
   #             name='norm1')
#    x= tf.nn.dropout(x,keep_prob)

    x = conv_layer(x, num_channels_out=64, spatial_stride=1,filterSize=3,name="Conv2_3x3")
    x = tf.nn.relu(x,name="relu2")
   # x = tf.nn.lrn(x, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
    #                name='norm2')
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    #x= tf.nn.dropout(x,keep_prob)
    x = conv_layer(x, num_channels_out=128, spatial_stride=1,filterSize=3,name="Conv3_3x3")
    x = tf.nn.relu(x,name="relu3")

    x = conv_layer(x, num_channels_out=256, spatial_stride=1,filterSize=3,name="Conv4_3x3")
    x = tf.nn.relu(x,name="relu4")
   # x = tf.nn.lrn(x, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
    #                name='norm2')
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1], padding='SAME', name='pool3')
 #   x= tf.nn.dropout(x,keep_prob)
    x = tf.reshape(x, [batch_size, -1])
    x = tf.nn.relu(x,name="relu5")
    x = linear_layer(x, num_outputs=1024,name="FC1_1024")
    x = tf.nn.relu(x,name="relu6")
    x= tf.nn.dropout(x,keep_prob,name="dropout_0.5")
    x = linear_layer(x, num_outputs=2048,name="FC2_2048")
    #x = tf.nn.relu(x)
    x = linear_layer(x, num_outputs=1024,name="FC3_1024")
    #x = tf.nn.relu(x)
  #  x= tf.nn.dropout(x,keep_prob)

    logits = linear_layer(x, num_outputs=NUM_CLASSES,name="output")

    return logits



In [4]:
batch_size=100
WIDTH = 64
HEIGHT = 64
CHANNELS = 3
OUTPUT=2

In [5]:
inputs = tf.placeholder(tf.float32,name='input',shape=[batch_size,WIDTH,HEIGHT,CHANNELS])
labels = tf.placeholder(tf.int32,name='target',shape=[batch_size,OUTPUT])
keep_prob = tf.placeholder(tf.float32) #for dropout
logits = buildNetwork(inputs, batch_size,keep_prob=keep_prob)
model = tf.global_variables_initializer()
sess.run(model)

In [6]:
writer = tf.summary.FileWriter('log')


In [7]:
writer.add_graph(sess.graph)